로지스틱 회귀 앙상블 실습

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

데이터셋, 데이터 로더 생성

In [ ]:
dataset = FashionMNIST(root='./data', train=True, transform=ToTensor(), download=False)
train_set, val_set = train_test_split(dataset, test_size=0.1, shuffle=777)

train_loader = DataLoader(train_set, batch_size=100, shuffle=True)
test_loader = DataLoader(val_set, batch_size=100, shuffle=False)

모델 선언

In [ ]:
class LogisitcRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisitcRegression, self).__init__()
        self.linear = nn.Linear(input_size,num_classes)
        
    def forward(self,x):
        out = self.linear(x)
        return out

하이퍼 파라미터 설정

In [ ]:
input_size = 28*28
num_classes = 10
num_epochs = 100
lr = 0.001
num_models = 5  # 앙상블에 사용할 모델 개수

앙상블

In [ ]:
models = [LogisitcRegression(input_size, num_classes) for _ in range(num_models)]
print(models)

모델, 손실 함수, 옵티마이저 정의

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizers = [optim.SGD(model.parameters(), lr=lr) for model in models]
print(models)

train loop

In [ ]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 데이터 로드
        images = images.reshape(-1,input_size)
        labels = labels
        # print(images,labels)
        
        # 순전파 및 손실 계산
        for j in range(num_models) :
            outputs = models[j](images)
            loss = criterion(outputs, labels)
            
            # 역전파 및 가중치 업데이트
            optimizers[j].zero_grad()
            loss.backward()
            optimizers[j].step()
            
        # 검증 코드 추가 #
        with torch.no_grad():
            total, correct = 0,0
            for images, labels in test_loader :
                images = images.reshape(-1,input_size)
                outputs = torch.zeros(images.size()[0], num_classes) # 초기화를 통해 업데이트
            """
            outputs = torch.zeros(images.size()[0], num_classes)
            이미지 배치에 대한 출력 텐서 초기화 
            >> 후속 단계에서 이미지에 대한 예측값 업데이트 가능
            """
            
                # 앙상블 모델의 예측값  더하기
            for j in range(num_models):
                outputs += models[j](images)
                
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            val_acc = 100 * correct / total
            print(f"Epoch [{epoch+1}/{num_epochs}], Val ACC >> {val_acc:.2f}%")